In [17]:
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv("../.env")

url = os.getenv("LOCAL_LESSON6_DS")

df = pd.read_csv(url)

In [18]:
# Убрать NaN
df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
df['HasCabin'] = df['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
# Бесполезное
df = df.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'])
# OHE
df['Sex'] = df['Sex'].apply(lambda x: 0 if x == 'male' else 1)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix = 'Em')], axis=1)
df = pd.concat([df, pd.get_dummies(df['Sex'], prefix = 'Sex')], axis=1)
df = pd.concat([df, pd.get_dummies(df['Pclass'], prefix = 'Pclass')], axis=1)
df = df.drop(columns = ['Embarked', 'Sex'])
df = df.drop(columns = ['Em_C', 'Sex_0', 'Pclass_1'])
# Один признак вместо двух
df['famsize'] = df['SibSp'] + df['Parch']
df = df.drop(columns = ['SibSp', 'Parch'])
# Копия, чтобы не трогать оригинал
df_nrml = df.copy()
# Нормализация
df_nrml['Age_n'] = (df_nrml['Age'] - df_nrml['Age'].min())/(df_nrml['Age'].max() - df_nrml['Age'].min())
df_nrml['Fare_n'] = (df_nrml['Fare'] - df_nrml['Fare'].min())/(df_nrml['Fare'].max() - df_nrml['Fare'].min())
df_nrml['famsize_n'] = (df_nrml['famsize'] - df_nrml['famsize'].min())/(df_nrml['famsize'].max() - df_nrml['famsize'].min())
df_nrml = df_nrml.drop(columns = ['Age', 'famsize', 'Fare'])
# Делим
x = df_nrml.drop(columns = ['Survived'])
y = df_nrml['Survived']

In [19]:
y = y.apply(lambda x: -1 if x == 0 else 1)

In [20]:
import random

w = [random.uniform(-0.01, 0.01) for i in range(x.shape[1])]

In [21]:
b = random.uniform(-0.01, 0.01)

In [81]:
L2 = 0.001
epochs = 6000
lr = 0.001

In [23]:
xl = x.values.tolist()
yl = y.values.tolist()

In [86]:
def pred(x, w, b):
    return sum([w[i] * x[i] for i in range(len(x))]) + b

In [90]:
def learn(xl, yl, w, b, lr, epoch):
    for epoch in range(epochs):
        y_p = []
        for i in range(len(xl)):
            a = pred(xl[i], w, b)
            y_p.append(a)

            if yl[i] * a >= 1:
                for j in range(len(xl[i])):
                    w[j] -= lr * 2 * L2 * w[j]
            else:
                for j in range(len(xl[i])):
                    w[j] -= lr *  (2 * L2 * w[j] - yl[i] * xl[i][j])
                b += lr * yl[i]
    
    return w, b

In [26]:
split = int(0.8 * len(xl))

xl_train = xl[:split]
yl_train = yl[:split]

xl_test = xl[split:]
yl_test = yl[split:]

In [91]:
learn(xl_train, yl_train, w, b, lr, epoch)

([-0.004178063150986046,
  0.026411605705667907,
  -0.009887187626282809,
  -0.009991202981521514,
  2.0093151006403454,
  0.013284560280055828,
  -0.011911740788230409,
  -0.06432651838863092,
  0.019902442561607944,
  -0.10109090307790286],
 -0.9547868949416789)

In [95]:
def accuracy(xl, yl, w, b):
    y_pred = [(1 if pred(i, w, b) >=0 else -1) for i in xl]

    acc = sum(y_pred[i] == yl[i] for i in range(len(y_pred)))/len(y_pred)
    return acc

In [96]:
accuracy(xl_train, yl_train, w, b)

0.7837078651685393

In [98]:
accuracy(xl_test, yl_test, w, b)

0.7988826815642458